# Create Voronoi from points
This notebook creates a Voronoi polygon mesh out of a set of points, including inifinite vertices
### Author
- Rubén Crespo Ceballos
## Outline
1. Open the points set. 
2. Create the Voronoi points, areas.
3. Convert them to polygons.
4. Clip them to the bounding box / AOI.
5. Append the info of the point to the polygon.
6. Export the result.


In [1]:
import geopandas as gpd
import numpy as np
from scipy.spatial import Voronoi
from shapely.geometry import Polygon, box, Point, MultiPolygon
from shapely.ops import unary_union


In [10]:
def voronoi_polygons(vor, points_array):
    """
    Generate Voronoi polygons from Voronoi diagram.

    Parameters:
    vor (Voronoi): A Voronoi object from scipy.spatial
    points_array (np.array): Array of points used to generate the Voronoi diagram

    Returns:
    list: List of Shapely Polygon objects representing the Voronoi regions
    """
    new_regions = []
    failed_regions = []
    valid_regions = []
    # Iterate over each point and its corresponding Voronoi region index
    for point_idx, region_idx in enumerate(vor.point_region):
        region = vor.regions[region_idx]
        # If the region does not contain an infinite vertex (-1) and has vertices
        if not -1 in region and len(region) > 0:
            polygon = Polygon([vor.vertices[i] for i in region]) # Create the polygon
            if polygon.is_valid:
                new_regions.append(polygon)
                valid_regions.append(polygon)
        else:
            # Handle regions with infinite vertices
            finite_region = [vor.vertices[i] for i in region if i != -1]
            if len(finite_region) > 2: # Ensure enough vertices to form a polygon
                finite_polygon = Polygon(finite_region)
                if finite_polygon.is_valid:
                    new_regions.append(finite_polygon)
                    valid_regions.append(polygon)
            else:
                # Store the failed polygons. 
                failed_regions.append(finite_region)

    if failed_regions: # This is very unlickely to happen, but just in case
        print("There are failed regions")
        # Create union of all valid polygons to find the covered area
        covered_area = unary_union(valid_regions)
        
        # Define a sufficiently large bounding box around the points
        min_x, min_y = points_array.min(axis=0) - 1
        max_x, max_y = points_array.max(axis=0) + 1
        bounding_box = Polygon([(min_x, min_y), (min_x, max_y), (max_x, max_y), (max_x, min_y)])
        
        # Calculate the empty area
        empty_area = bounding_box.difference(covered_area)
        
        # Add buffer polygons clipped to the empty area
        for point_idx, region_idx in enumerate(vor.point_region):
            region = vor.regions[region_idx]
            if -1 in region or len(region) <= 2:
                buffer_polygon = Point(points_array[point_idx]).buffer(0.1) # Adjust the buffer to the projection type
                clipped_polygon = buffer_polygon.intersection(empty_area)
                if clipped_polygon.is_valid and not clipped_polygon.is_empty:
                    new_regions.append(clipped_polygon)
    
    return new_regions, failed_regions

In [11]:
# Load the points shapefile
gdf = gpd.read_file(r"Z:\z_resources\ruben\tin_buildings\filtered_points.shp")  # Replace with the correct path to your shapefile
points = np.array(list(zip(gdf.geometry.x, gdf.geometry.y)))

# Generate Voronoi diagram
vor = Voronoi(points)

# Generate Voronoi polygons
voronoi_polys, failed_regions = voronoi_polygons(vor, points)

if failed_regions:
    print("We have a problem here")

In [12]:
# Define the bounding box around the points
minx, miny, maxx, maxy = gdf.total_bounds
additional_units = 0.001
aoi = box(minx - additional_units, miny - additional_units, maxx + additional_units, maxy + additional_units)

In [ ]:
# If I want to use the linestring of a multypolygon
aoi_gdf = gpd.read_file('path_to_your_aoi_file.shp')

# Ensure AOI is a MultiPolygon
if isinstance(aoi_gdf.geometry.iloc[0], MultiPolygon):
    aoi = aoi_gdf.geometry.iloc[0]
else:
    aoi = unary_union(aoi_gdf.geometry)

In [ ]:
# Clip the polygons to the bounding box (This will be changed with a vector intput of the AOI)
clipped_polygons = [poly.intersection(aoi) for poly in voronoi_polys]

In [13]:
"""Pass the points attribute info to the polygons 1-1"""
# Ensure the number of polygons matches the number of points
assert len(clipped_polygons) == len(points), "Number of polygons does not match number of points"

# Create a GeoDataFrame for the Voronoi polygons
voronoi_gdf = gpd.GeoDataFrame(geometry=clipped_polygons, crs=gdf.crs)

# Add the original point attributes to the Voronoi polygons
joined_gdf = voronoi_gdf.join(gdf.drop(columns='geometry')) # drop the geometry of the points

In [14]:
# Export the final GeoDataFrame
joined_gdf.to_file(r"Z:\z_resources\ruben\tin_buildings\voronoi_test_no_buffer.shp")